# Spam/Ham classification Model

In [1]:
# read file into pandas using a relative path
import pandas as pd
path = 'data/spam.tsv'
df = pd.read_table(path, header=None, names=['label', 'message'])

In [2]:
# examine the shape
df.shape

(5572, 2)

In [3]:
# examine the first 10 rows
df.head(10)

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [4]:
# examine the class distribution
df.label.value_counts(normalize=True)

ham     0.865937
spam    0.134063
Name: label, dtype: float64

In [5]:
# Cleaning the texts
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [6]:
stemmer = PorterStemmer()

In [8]:
for i in range(len(df)):
    sms = re.sub('[^a-zA-Z]', ' ', df.message[i])
    sms = sms.lower()
    sms = sms.split()
    sms = [stemmer.stem(word) for word in sms if not word in set(stopwords.words('english'))]
    sms = ' '.join(sms)
    df.message[i] = sms

In [9]:
df.head(10)

,label,message
0,ham,go jurong point crazi avail bugi n great world...
1,ham,ok lar joke wif u oni
2,spam,free entri wkli comp win fa cup final tkt st m...
3,ham,u dun say earli hor u c alreadi say
4,ham,nah think goe usf live around though
5,spam,freemsg hey darl week word back like fun still...
6,ham,even brother like speak treat like aid patent
7,ham,per request mell mell oru minnaminungint nurun...
8,spam,winner valu network custom select receivea pri...
9,spam,mobil month u r entitl updat latest colour mob...


In [10]:
# convert label to a numerical variable
df['label_num'] = df.label.map({'ham':0, 'spam':1})

In [11]:
# check that the conversion worked
df.head(10)

,label,message,label_num
0,ham,go jurong point crazi avail bugi n great world...,0
1,ham,ok lar joke wif u oni,0
2,spam,free entri wkli comp win fa cup final tkt st m...,1
3,ham,u dun say earli hor u c alreadi say,0
4,ham,nah think goe usf live around though,0
5,spam,freemsg hey darl week word back like fun still...,1
6,ham,even brother like speak treat like aid patent,0
7,ham,per request mell mell oru minnaminungint nurun...,0
8,spam,winner valu network custom select receivea pri...,1
9,spam,mobil month u r entitl updat latest colour mob...,1


In [12]:
# how to define X and y (from the SMS data) for use with COUNTVECTORIZER
X = df.message
y = df.label_num
print(X.shape)
print(y.shape)

(5572,)
(5572,)


Notice that X is one dimensional here. When we pass it to count vectorizer, it will convert it to two dimensional matrix. 

In [13]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4179,)
(1393,)
(4179,)
(1393,)


In [14]:
# import and instantiate CountVectorizer (with the default parameters)
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [15]:
X_train_dtm = cv.fit_transform(X_train)

In [16]:
# examine the document-term matrix
X_train_dtm

<4179x5409 sparse matrix of type '<class 'numpy.int64'>'
	with 33627 stored elements in Compressed Sparse Row format>

In [17]:
# transform testing data (using fitted vocabulary) into a document-term matrix
X_test_dtm = cv.transform(X_test)
X_test_dtm

<1393x5409 sparse matrix of type '<class 'numpy.int64'>'
	with 10548 stored elements in Compressed Sparse Row format>

## Multinomial Naive Bayes

In [18]:
# import and instantiate a Multinomial Naive Bayes model
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

In [19]:
# train the model using X_train_dtm (timing it with an IPython "magic command")
nb.fit(X_train_dtm, y_train)

MultinomialNB()

In [20]:
# make class predictions for X_test_dtm
y_pred_class = nb.predict(X_test_dtm)

In [21]:
# calculate accuracy of class predictions
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score
accuracy_score(y_test, y_pred_class)

0.9899497487437185

In [22]:
nb.classes_

array([0, 1], dtype=int64)

In [23]:
# print the confusion matrix
confusion_matrix(y_test, y_pred_class)

array([[1202,    6],
       [   8,  177]], dtype=int64)

In [24]:
nb.predict_proba(X_test_dtm)

array([[9.93224980e-01, 6.77501978e-03],
       [9.99277766e-01, 7.22233679e-04],
       [9.00215562e-01, 9.97844379e-02],
       ...,
       [9.99993384e-01, 6.61590364e-06],
       [1.77471480e-10, 1.00000000e+00],
       [9.99999930e-01, 6.97910991e-08]])

The column on the left represents the probability the message is of class 0, and the column on the right represents the probability the message is of class 1.

In [25]:
# calculate predicted probabilities for X_test_dtm (poorly calibrated)
y_pred_prob = nb.predict_proba(X_test_dtm)[:, 1]
y_pred_prob

array([6.77501978e-03, 7.22233679e-04, 9.97844379e-02, ...,
       6.61590364e-06, 1.00000000e+00, 6.97910991e-08])

Understand that naive bayes predicts very extreme value and there probability provided by it should not be understood as probability in colloquial definition. It just performed some mathematical operation and gave out the numbers. In simple words, the predicted probability provided by naive bayes is not well caliberated.

In [26]:
# calculate AUC
roc_auc_score(y_test, y_pred_prob)

0.9894531949167712